In [1]:
%load_ext autoreload
%autosave 60
%autoreload 2

Autosaving every 60 seconds


In [2]:
from google.colab import drive
drive.mount("/content/drive")
%cd drive/MyDrive/FusionModel

Mounted at /content/drive
/content/drive/MyDrive/FusionModel


In [4]:
# !pip install torch==1.9.0 torchvision==0.10.0 torchaudio==0.9.0 torchtext==0.10.0
!pip install transformers==4.8.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.5 MB 6.8 MB/s 
     |████████████████████████████████| 880 kB 25.8 MB/s 
     |████████████████████████████████| 3.3 MB 34.5 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=f75b94ada3b04aa41a4ed5f7e4e8f7df42ec210bbcf9bbd7bf3c2c0a747ffe0f
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses


In [6]:
splits = [10,20,30]
num_classes = len(splits)

In [7]:
from dataset import YouTubeDataset
dataset = YouTubeDataset(splits)

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [8]:
from model import SimpleBert
model = SimpleBert(num_classes)

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Initialization success if you see a tensor: tensor([[-0.1859,  0.0087, -0.1187]], grad_fn=<AddmmBackward0>).


In [ ]:
# import torch
# checkpoint = torch.load("checkpoints/epoch200.pt",map_location=torch.device('cpu'))
# model.load_state_dict(checkpoint['model_state_dict'])


In [ ]:
# model.base.requires_grad = False

In [4]:
import torch
import math
import os
import time
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
from train import eval, get_scores
from torch.nn.functional import cross_entropy

def train_model(model, dataset, learning_rate, lr_decay, batch_size, num_epochs, device='cuda', isCheckpoint=False, train_val_split = None, isVerbose=True):
    loss_history = []

    model = model.to(device)
    model.train()

    optimizer = torch.optim.AdamW(
        filter(lambda p: p.requires_grad, model.parameters()), learning_rate
    )
    lr_scheduler = torch.optim.lr_scheduler.LambdaLR(
        optimizer, lambda epoch: lr_decay ** epoch
    )

    # sample minibatch data
    if not train_val_split:
      train_ids = [i for i in range(len(dataset))]
      val_ids = None
    else:
      train_ids, val_ids = train_val_split

    iter_per_epoch = math.ceil(len(train_ids) // batch_size)
    class_weights = torch.tensor(compute_class_weight(class_weight='balanced', classes=torch.unique(dataset.label[train_ids]).numpy(), y=dataset.label[train_ids].numpy()), dtype=torch.float, device=device)
    loss_fn = torch.nn.NLLLoss(weight = class_weights)
    # loss_fn = cross_entropy
    
    for i in range(num_epochs):
        start_t = time.time()
        local_hist = []
        correct_cnt = 0
        y_preds = torch.empty((0,),device=device)
        y_trues = torch.empty((0,),device=device)
        for j in range(iter_per_epoch):
            tokens, y_true = dataset[train_ids[j * batch_size: (j + 1) * batch_size]]

            tokens = tokens.to(device)
            y_true = y_true.to(device)

            optimizer.zero_grad()

            digits = model(tokens)
            y_preds = torch.hstack([y_preds,digits.argmax(dim=1)])
            y_trues = torch.hstack([y_trues,y_true])

            probs = torch.nn.LogSoftmax(dim=1)(digits)
            loss = loss_fn(probs,y_true)
            loss.backward()

            local_hist.append(loss.item())
            optimizer.step()

        end_t = time.time()

        loss_mean = np.array(local_hist).mean()
        loss_history.append(loss_mean)
            
        print(
            f"(Epoch {i} / {num_epochs}), time: {end_t - start_t:.1f}s, loss: {loss_mean:.3f}"
        )
        if isVerbose:
            train_accuracy, train_precision, train_recall, train_f1 = get_scores(y_trues.to('cpu'), y_preds.to('cpu'), model.num_classes) # This is an aggregated result due to GPU size limit
            print(f"    Training Set - accuracy: {train_accuracy:.2f}, precision: {train_precision:.2f}, recall: {train_recall:.2f}, f1-score: {train_f1:.2f},")
            if val_ids is not None:
                val_accuracy, val_precision, val_recall, val_f1 = eval(model, dataset, val_ids, num_classes)
                print(f"    Validation Set - accuracy: {val_accuracy:.2f}, precision: {val_precision:.2f}, recall: {val_recall:.2f}, f1-score: {val_f1:.2f},")
        if i%200 == 0 and isCheckpoint:
          dir = "checkpoints"
          if not os.path.exists(dir):
            os.mkdir(dir)
          file = f"epoch{i}.pt"
          path = dir+'/'+file
          torch.save({
                      'epoch': i,
                      'model_state_dict': model.state_dict(),
                      'optimizer_state_dict': optimizer.state_dict(),
                      'loss': loss_mean,
                      }, path)

        lr_scheduler.step()
    
    return loss_history

In [ ]:
loss_hist = train_model(model, dataset, learning_rate=5e-6, lr_decay=0.99, batch_size=10, num_epochs=200, isCheckpoint = True, isVerbose = True)

## 5-fold CV

In [5]:
from sklearn.model_selection import KFold
import torch

def train_model_cv5(model, dataset):
    loss_hist = []
    kf = KFold(n_splits=5)
    cnt = 1
    for train_index, val_index in kf.split(dataset):
        model.reset()
        print("Fold "+str(cnt)+" (val", val_index[0],"-",str(val_index[-1])+")")
        loss_hist_fold = train_model(model, dataset=dataset, train_val_split=(train_index, val_index),learning_rate=3e-6, lr_decay=0.99, batch_size=10, num_epochs=200, isCheckpoint = False, isVerbose = True)
        loss_hist.append(loss_hist_fold)
        cnt += 1
    return loss_hist

In [ ]:
lost_hist_folds = train_model_cv5(model, dataset)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Initialization success if you see a tensor: tensor([[0.2294, 0.2600, 0.0036, 0.0852]], grad_fn=<AddmmBackward>).
Fold 1 (val 0 - 42)
(Epoch 0 / 200), time: 9.2s, loss: 1.385
    Training Set - accuracy: 0.35, precision: 0.35, recall: 0.35, f1-score: 0.35,
    Validation Set - accuracy: 0.12, precision: 0.12, recall: 0.12, f1-score: 0.12,
(Epoch 1 / 200), time: 9.2s, loss: 1.381
    Training Set - accuracy: 0.35, precision: 0.35, recall: 0.35, f1-score: 0.35,
    Validation Set - accuracy: 0.12, precision: 0.12, recall: 0.12, f1-score: 0.12,
(Epoch 2 / 200), time: 9.2s, loss: 1.382
    Training Set - accuracy: 0.35, precision: 0.35, recall: 0.35, f1-score: 0.35,
    Validation Set - accuracy: 0.12, precision: 0.12, recall: 0.12, f1-score: 0.12,
(Epoch 3 / 200), time: 9.1s, loss: 1.381
    Training Set - accuracy: 0.35, precision: 0.35, recall: 0.35, f1-score: 0.35,
    Validation Set - accuracy: 0.12, precision: 0.12, recall: 0.12, f1-score: 0.12,
(Epoch 4 / 200), time: 9.2s, loss: 1.38

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Initialization success if you see a tensor: tensor([[ 0.0452,  0.1372, -0.2984, -0.1721]], grad_fn=<AddmmBackward>).
Fold 2 (val 43 - 84)
(Epoch 0 / 200), time: 9.2s, loss: 1.388
    Training Set - accuracy: 0.33, precision: 0.33, recall: 0.33, f1-score: 0.33,
    Validation Set - accuracy: 0.40, precision: 0.40, recall: 0.40, f1-score: 0.40,
(Epoch 1 / 200), time: 9.2s, loss: 1.382
    Training Set - accuracy: 0.36, precision: 0.36, recall: 0.36, f1-score: 0.36,
    Validation Set - accuracy: 0.25, precision: 0.25, recall: 0.25, f1-score: 0.25,
(Epoch 2 / 200), time: 9.2s, loss: 1.385
    Training Set - accuracy: 0.35, precision: 0.35, recall: 0.35, f1-score: 0.35,
    Validation Set - accuracy: 0.28, precision: 0.28, recall: 0.28, f1-score: 0.28,
(Epoch 3 / 200), time: 9.2s, loss: 1.390
    Training Set - accuracy: 0.30, precision: 0.30, recall: 0.30, f1-score: 0.30,
    Validation Set - accuracy: 0.23, precision: 0.23, recall: 0.23, f1-score: 0.23,
(Epoch 4 / 200), time: 9.1s, loss:

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Initialization success if you see a tensor: tensor([[ 0.1924, -0.2593,  0.2054,  0.1718]], grad_fn=<AddmmBackward>).
Fold 3 (val 85 - 126)
(Epoch 0 / 200), time: 9.2s, loss: 1.408
    Training Set - accuracy: 0.35, precision: 0.35, recall: 0.35, f1-score: 0.35,
    Validation Set - accuracy: 0.25, precision: 0.25, recall: 0.25, f1-score: 0.25,
(Epoch 1 / 200), time: 9.2s, loss: 1.402
    Training Set - accuracy: 0.49, precision: 0.49, recall: 0.49, f1-score: 0.49,
    Validation Set - accuracy: 0.25, precision: 0.25, recall: 0.25, f1-score: 0.25,
(Epoch 2 / 200), time: 9.2s, loss: 1.403
    Training Set - accuracy: 0.36, precision: 0.36, recall: 0.36, f1-score: 0.36,
    Validation Set - accuracy: 0.28, precision: 0.28, recall: 0.28, f1-score: 0.28,
(Epoch 3 / 200), time: 9.2s, loss: 1.398
    Training Set - accuracy: 0.28, precision: 0.28, recall: 0.28, f1-score: 0.28,
    Validation Set - accuracy: 0.15, precision: 0.15, recall: 0.15, f1-score: 0.15,
(Epoch 4 / 200), time: 9.2s, loss

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Initialization success if you see a tensor: tensor([[-0.3458, -0.0814, -0.0866,  0.2677]], grad_fn=<AddmmBackward>).
Fold 4 (val 127 - 168)
(Epoch 0 / 200), time: 9.2s, loss: 1.448
    Training Set - accuracy: 0.06, precision: 0.06, recall: 0.06, f1-score: 0.06,
    Validation Set - accuracy: 0.05, precision: 0.05, recall: 0.05, f1-score: 0.05,
(Epoch 1 / 200), time: 9.2s, loss: 1.437
    Training Set - accuracy: 0.06, precision: 0.06, recall: 0.06, f1-score: 0.06,
    Validation Set - accuracy: 0.05, precision: 0.05, recall: 0.05, f1-score: 0.05,
(Epoch 2 / 200), time: 9.1s, loss: 1.435
    Training Set - accuracy: 0.06, precision: 0.06, recall: 0.06, f1-score: 0.06,
    Validation Set - accuracy: 0.05, precision: 0.05, recall: 0.05, f1-score: 0.05,
(Epoch 3 / 200), time: 9.2s, loss: 1.435
    Training Set - accuracy: 0.06, precision: 0.06, recall: 0.06, f1-score: 0.06,
    Validation Set - accuracy: 0.05, precision: 0.05, recall: 0.05, f1-score: 0.05,
(Epoch 4 / 200), time: 9.2s, los

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Initialization success if you see a tensor: tensor([[-0.0026,  0.1498, -0.1648, -0.2448]], grad_fn=<AddmmBackward>).
Fold 5 (val 169 - 210)
(Epoch 0 / 200), time: 9.2s, loss: 1.379
    Training Set - accuracy: 0.25, precision: 0.25, recall: 0.25, f1-score: 0.25,
    Validation Set - accuracy: 0.47, precision: 0.47, recall: 0.47, f1-score: 0.48,
(Epoch 1 / 200), time: 9.2s, loss: 1.378
    Training Set - accuracy: 0.25, precision: 0.25, recall: 0.25, f1-score: 0.25,
    Validation Set - accuracy: 0.47, precision: 0.47, recall: 0.47, f1-score: 0.48,
(Epoch 2 / 200), time: 9.2s, loss: 1.379
    Training Set - accuracy: 0.25, precision: 0.25, recall: 0.25, f1-score: 0.25,
    Validation Set - accuracy: 0.47, precision: 0.47, recall: 0.47, f1-score: 0.48,
(Epoch 3 / 200), time: 9.2s, loss: 1.379
    Training Set - accuracy: 0.25, precision: 0.25, recall: 0.25, f1-score: 0.25,
    Validation Set - accuracy: 0.47, precision: 0.47, recall: 0.47, f1-score: 0.48,
(Epoch 4 / 200), time: 9.2s, los

In [8]:
from dataset import YouTubeDataset
from model import SimpleBert
splits = [10,20]
num_classes = len(splits)+1
dataset = YouTubeDataset(splits)
model = SimpleBert(num_classes)

hist = train_model_cv5(model, dataset)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Initialization success if you see a tensor: tensor([[0.1725, 0.3847, 0.0342]], grad_fn=<AddmmBackward>).


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Fold 1 (val 0 - 42)
(Epoch 0 / 200), time: 9.1s, loss: 1.109
    Training Set - accuracy: 0.21, precision: 0.21, recall: 0.21, f1-score: 0.21,
    Validation Set - accuracy: 0.23, precision: 0.23, recall: 0.23, f1-score: 0.23,
(Epoch 1 / 200), time: 9.0s, loss: 1.102
    Training Set - accuracy: 0.21, precision: 0.21, recall: 0.21, f1-score: 0.21,
    Validation Set - accuracy: 0.20, precision: 0.20, recall: 0.20, f1-score: 0.20,
(Epoch 2 / 200), time: 9.0s, loss: 1.109
    Training Set - accuracy: 0.29, precision: 0.29, recall: 0.29, f1-score: 0.29,
    Validation Set - accuracy: 0.40, precision: 0.40, recall: 0.40, f1-score: 0.40,
(Epoch 3 / 200), time: 9.0s, loss: 1.103
    Training Set - accuracy: 0.38, precision: 0.38, recall: 0.38, f1-score: 0.38,
    Validation Set - accuracy: 0.53, precision: 0.53, recall: 0.53, f1-score: 0.53,
(Epoch 4 / 200), time: 9.0s, loss: 1.103
    Training Set - accuracy: 0.42, precision: 0.42, recall: 0.42, f1-score: 0.42,
    Validation Set - accuracy

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


...

Fold 3 (val 85 - 126)
(Epoch 0 / 200), time: 9.1s, loss: 1.090
    Training Set - accuracy: 0.59, precision: 0.59, recall: 0.59, f1-score: 0.59,
    Validation Set - accuracy: 0.38, precision: 0.38, recall: 0.38, f1-score: 0.38,
(Epoch 1 / 200), time: 9.0s, loss: 1.089
    Training Set - accuracy: 0.59, precision: 0.59, recall: 0.59, f1-score: 0.59,
    Validation Set - accuracy: 0.38, precision: 0.38, recall: 0.38, f1-score: 0.38,
(Epoch 2 / 200), time: 9.0s, loss: 1.087
    Training Set - accuracy: 0.59, precision: 0.59, recall: 0.59, f1-score: 0.59,
    Validation Set - accuracy: 0.38, precision: 0.38, recall: 0.38, f1-score: 0.38,
(Epoch 3 / 200), time: 9.0s, loss: 1.088
    Training Set - accuracy: 0.59, precision: 0.59, recall: 0.59, f1-score: 0.59,
    Validation Set - accuracy: 0.38, precision: 0.38, recall: 0.38, f1-score: 0.38,
(Epoch 4 / 200), time: 9.0s, loss: 1.086
    Training Set - accuracy: 0.59, precision: 0.59, recall: 0.59, f1-score: 0.59,
    Validation Set - accura

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Fold 4 (val 127 - 168)
(Epoch 0 / 200), time: 9.1s, loss: 1.090
    Training Set - accuracy: 0.57, precision: 0.57, recall: 0.57, f1-score: 0.57,
    Validation Set - accuracy: 0.45, precision: 0.45, recall: 0.45, f1-score: 0.45,
(Epoch 1 / 200), time: 9.0s, loss: 1.086
    Training Set - accuracy: 0.57, precision: 0.57, recall: 0.57, f1-score: 0.57,
    Validation Set - accuracy: 0.45, precision: 0.45, recall: 0.45, f1-score: 0.45,
(Epoch 2 / 200), time: 9.0s, loss: 1.087
    Training Set - accuracy: 0.57, precision: 0.57, recall: 0.57, f1-score: 0.57,
    Validation Set - accuracy: 0.45, precision: 0.45, recall: 0.45, f1-score: 0.45,
(Epoch 3 / 200), time: 9.0s, loss: 1.088
    Training Set - accuracy: 0.57, precision: 0.57, recall: 0.57, f1-score: 0.57,
    Validation Set - accuracy: 0.45, precision: 0.45, recall: 0.45, f1-score: 0.45,
(Epoch 4 / 200), time: 9.0s, loss: 1.086
    Training Set - accuracy: 0.57, precision: 0.57, recall: 0.57, f1-score: 0.57,
    Validation Set - accur

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Fold 5 (val 169 - 210)
(Epoch 0 / 200), time: 9.1s, loss: 1.135
    Training Set - accuracy: 0.25, precision: 0.25, recall: 0.25, f1-score: 0.25,
    Validation Set - accuracy: 0.47, precision: 0.47, recall: 0.47, f1-score: 0.48,
(Epoch 1 / 200), time: 9.0s, loss: 1.126
    Training Set - accuracy: 0.25, precision: 0.25, recall: 0.25, f1-score: 0.25,
    Validation Set - accuracy: 0.47, precision: 0.47, recall: 0.47, f1-score: 0.48,
(Epoch 2 / 200), time: 9.0s, loss: 1.123
    Training Set - accuracy: 0.25, precision: 0.25, recall: 0.25, f1-score: 0.25,
    Validation Set - accuracy: 0.47, precision: 0.47, recall: 0.47, f1-score: 0.48,
(Epoch 3 / 200), time: 9.0s, loss: 1.116
    Training Set - accuracy: 0.25, precision: 0.25, recall: 0.25, f1-score: 0.25,
    Validation Set - accuracy: 0.47, precision: 0.47, recall: 0.47, f1-score: 0.48,
(Epoch 4 / 200), time: 9.0s, loss: 1.116
    Training Set - accuracy: 0.25, precision: 0.25, recall: 0.25, f1-score: 0.25,
    Validation Set - accur

In [19]:
dataset.label

tensor([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1,
        0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0,
        0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1,
        1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0,
        0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1,
        0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1,
        0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0])

In [7]:
from dataset import YouTubeDataset
from model import SimpleBert
splits = [10]
num_classes = len(splits)+1
dataset = YouTubeDataset(splits)
model = SimpleBert(num_classes)

hist = train_model_cv5(model, dataset)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Initialization success if you see a tensor: tensor([[-0.2806, -0.1921]], grad_fn=<AddmmBackward>).


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Fold 1 (val 0 - 42)
(Epoch 0 / 200), time: 9.1s, loss: 0.711
    Training Set - accuracy: 0.53, precision: 0.53, recall: 0.53, f1-score: 0.53,
    Validation Set - accuracy: 0.17, precision: 0.17, recall: 0.17, f1-score: 0.17,
(Epoch 1 / 200), time: 9.1s, loss: 0.697
    Training Set - accuracy: 0.54, precision: 0.54, recall: 0.54, f1-score: 0.54,
    Validation Set - accuracy: 0.17, precision: 0.17, recall: 0.17, f1-score: 0.17,
(Epoch 2 / 200), time: 9.0s, loss: 0.697
    Training Set - accuracy: 0.54, precision: 0.54, recall: 0.54, f1-score: 0.54,
    Validation Set - accuracy: 0.23, precision: 0.23, recall: 0.23, f1-score: 0.23,
(Epoch 3 / 200), time: 9.0s, loss: 0.697
    Training Set - accuracy: 0.53, precision: 0.53, recall: 0.53, f1-score: 0.53,
    Validation Set - accuracy: 0.28, precision: 0.28, recall: 0.28, f1-score: 0.28,
(Epoch 4 / 200), time: 9.0s, loss: 0.692
    Training Set - accuracy: 0.54, precision: 0.54, recall: 0.54, f1-score: 0.54,
    Validation Set - accuracy

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


...

Fold 3 (val 85 - 126)
(Epoch 0 / 200), time: 9.1s, loss: 0.713
    Training Set - accuracy: 0.41, precision: 0.41, recall: 0.41, f1-score: 0.41,
    Validation Set - accuracy: 0.62, precision: 0.62, recall: 0.62, f1-score: 0.62,
(Epoch 1 / 200), time: 9.0s, loss: 0.714
    Training Set - accuracy: 0.41, precision: 0.41, recall: 0.41, f1-score: 0.41,
    Validation Set - accuracy: 0.65, precision: 0.65, recall: 0.65, f1-score: 0.65,
(Epoch 2 / 200), time: 9.0s, loss: 0.709
    Training Set - accuracy: 0.41, precision: 0.41, recall: 0.41, f1-score: 0.41,
    Validation Set - accuracy: 0.60, precision: 0.60, recall: 0.60, f1-score: 0.60,
(Epoch 3 / 200), time: 9.0s, loss: 0.708
    Training Set - accuracy: 0.41, precision: 0.41, recall: 0.41, f1-score: 0.41,
    Validation Set - accuracy: 0.65, precision: 0.65, recall: 0.65, f1-score: 0.65,
(Epoch 4 / 200), time: 9.1s, loss: 0.707
    Training Set - accuracy: 0.39, precision: 0.39, recall: 0.39, f1-score: 0.39,
    Validation Set - accura

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Fold 4 (val 127 - 168)
(Epoch 0 / 200), time: 9.1s, loss: 0.694
    Training Set - accuracy: 0.57, precision: 0.57, recall: 0.57, f1-score: 0.57,
    Validation Set - accuracy: 0.45, precision: 0.45, recall: 0.45, f1-score: 0.45,
(Epoch 1 / 200), time: 9.0s, loss: 0.693
    Training Set - accuracy: 0.57, precision: 0.57, recall: 0.57, f1-score: 0.57,
    Validation Set - accuracy: 0.45, precision: 0.45, recall: 0.45, f1-score: 0.45,
(Epoch 2 / 200), time: 9.0s, loss: 0.691
    Training Set - accuracy: 0.57, precision: 0.57, recall: 0.57, f1-score: 0.57,
    Validation Set - accuracy: 0.45, precision: 0.45, recall: 0.45, f1-score: 0.45,
(Epoch 3 / 200), time: 9.0s, loss: 0.691
    Training Set - accuracy: 0.57, precision: 0.57, recall: 0.57, f1-score: 0.57,
    Validation Set - accuracy: 0.45, precision: 0.45, recall: 0.45, f1-score: 0.45,
(Epoch 4 / 200), time: 9.0s, loss: 0.691
    Training Set - accuracy: 0.57, precision: 0.57, recall: 0.57, f1-score: 0.57,
    Validation Set - accur

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Fold 5 (val 169 - 210)
(Epoch 0 / 200), time: 9.1s, loss: 0.704
    Training Set - accuracy: 0.60, precision: 0.60, recall: 0.60, f1-score: 0.60,
    Validation Set - accuracy: 0.35, precision: 0.35, recall: 0.35, f1-score: 0.35,
(Epoch 1 / 200), time: 9.0s, loss: 0.700
    Training Set - accuracy: 0.60, precision: 0.60, recall: 0.60, f1-score: 0.60,
    Validation Set - accuracy: 0.35, precision: 0.35, recall: 0.35, f1-score: 0.35,
(Epoch 2 / 200), time: 9.0s, loss: 0.699
    Training Set - accuracy: 0.60, precision: 0.60, recall: 0.60, f1-score: 0.60,
    Validation Set - accuracy: 0.35, precision: 0.35, recall: 0.35, f1-score: 0.35,
(Epoch 3 / 200), time: 9.0s, loss: 0.697
    Training Set - accuracy: 0.60, precision: 0.60, recall: 0.60, f1-score: 0.60,
    Validation Set - accuracy: 0.35, precision: 0.35, recall: 0.35, f1-score: 0.35,
(Epoch 4 / 200), time: 9.0s, loss: 0.697
    Training Set - accuracy: 0.60, precision: 0.60, recall: 0.60, f1-score: 0.60,
    Validation Set - accur